<a href="https://colab.research.google.com/github/mahmoud-elsh/ngo_datahelpers/blob/main/Final_Group3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pyngrok reportlab python-docx docx pycountry langchain_community chromadb langchain_huggingface matplotlib deepl

In [ ]:
%%writefile llama_backend.py
import torch, pandas as pd, requests, pycountry
from bs4 import BeautifulSoup
from transformers import pipeline, AutoTokenizer
from huggingface_hub import login
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from google.colab import userdata
import deepl
import os

_llama_pipeline = None
_base_prompt = None
_embedding_model = None
_llama_pipeline2 = None

def init_backend():
    global _llama_pipeline, _base_prompt, _embedding_model, _llama_pipeline2

    if _llama_pipeline is not None and _embedding_model is not None and _llama_pipeline2 is not None:
        return

    HF_TOKEN = os.getenv("HF_TOKEN")
    if HF_TOKEN:
        login(HF_TOKEN)

    _base_prompt = """
You are an expert humanitarian data analyst working for an NGO.

TASK:
Analyze socioeconomic and humanitarian data provided from multiple datasets and produce concise, structured insights.

STRICT RULES:
1. Ground every statement strictly in the provided data. Never guess or infer missing information.
2. Ignore and exclude any NaN, null, or missing values completely.
3. Discuss only data related to the specified country.
4. Do NOT give any recommendations, opinions, or projections.
5. Only Include citations inline using this exact format:
   - (World Bank)
   - (ReliefWeb)
   - (Google News)
   - (ACLED)
   Never include the sources at the end of your response.
6. Keep the tone formal, analytical, and data-driven.
7. Do not repeat any data or information.
8. Keep a consistent font and formatting.
9. Do not output sources at the end, only inline.

OUTPUT FORMAT:
**Executive Summary**
- Provide a concise overview (1–2 paragraphs) summarizing key humanitarian and socioeconomic conditions.

**Key Events**
- Summarize 4–6 major factual developments (conflict, migration, disasters, economic shifts, disease, immigration, or war) with inline citations.

**Trends**
- Discuss observable trends in population, economy, displacement, and aid response, grounded in the data.

**Risks**
- Identify emerging or ongoing humanitarian risks (conflict escalation, funding gaps, or instability) strictly from the data.
"""

    model = "meta-llama/Llama-3.2-3B-Instruct"
    tokenizer = AutoTokenizer.from_pretrained(model)

    _llama_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        dtype=torch.bfloat16,
        device_map="auto",
        do_sample=False,
        repetition_penalty=1.15,
        return_full_text=False,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=1024
    )

    model2 = "meta-llama/Llama-3.1-8B-Instruct"
    tokenizer = AutoTokenizer.from_pretrained(model2)

    _llama_pipeline2 = pipeline(
        "text-generation",
        model=model2,
        tokenizer=tokenizer,
        dtype=torch.bfloat16,
        device_map="auto",
        do_sample=False,
        repetition_penalty=1.15,
        return_full_text=False,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=1024
    )

    from langchain_huggingface import HuggingFaceEmbeddings
    _embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


def fetch_worldbank(country, indicator, start, end):
    url = f"https://api.worldbank.org/v2/country/{country}/indicator/{indicator}"
    params = {"date": f"{start}:{end}", "format": "json"}
    response = requests.get(url, params)
    if response.status_code != 200:
        return pd.DataFrame()
    response = response.json()
    if len(response) < 2 or response[1] is None:
        return pd.DataFrame()
    df = pd.json_normalize(response[1])
    if df.empty:
        return df
    return df[["indicator.id", "indicator.value", "date", "value"]]

def fetch_indicators(country, start, end):
    indicators = {
        "SP.POP.TOTL": "Total population",
        "SP.POP.GROW": "Population growth (annual %)",
        "NY.GDP.MKTP.CD": "GDP (current US$)",
        "NY.GDP.MKTP.KD.ZG": "GDP growth (annual %)",
        "NY.GDP.PCAP.CD": "GDP per capita (current US$)",
        "SL.UEM.TOTL.ZS": "Unemployment, total (% of labor force)",
        "FP.CPI.TOTL.ZG": "Inflation, consumer prices (annual %)",
    }
    dfs = []
    for i in indicators.keys():
        df = fetch_worldbank(country, i, start, end)
        if df is None or df.empty:
            continue
        dfs.append(df)
    if not dfs:
        return pd.DataFrame()
    df = pd.concat(dfs)
    df = df.pivot(index="date", columns="indicator.id", values="value")
    df = df.rename(columns=indicators)
    return df

def create_prompt_worldbank(country, start, end):
    df = fetch_indicators(country, start, end)
    if df is None or df.empty:
        return f"No World Bank data found for {country} between {start} and {end}."
    # Show the latest row if multiple dates
    try:
        last = df.sort_index().iloc[-1]
    except:
        return f"No World Bank data found for {country} between {start} and {end}."
    prompt = f""" This is data for {country} in {end} from the World Bank API:
  - Total Population: {last.get('Total population')}
  - Population Growth: {last.get('Population growth (annual %)')}
  - GDP: {last.get('GDP (current US$)')}
  - GDP Growth: {last.get('GDP growth (annual %)')}
  - GDP per Capita: {last.get('GDP per capita (current US$)')}
  - Unemployment: {last.get('Unemployment, total (% of labor force)')}
  - Inflation: {last.get('Inflation, consumer prices (annual %)')}
  - If any data is NaN, do not include it in the summary.
  """
    return prompt

def clean_html(input):
    if not input:
        return ""
    soup = BeautifulSoup(input, "html.parser")
    for br in soup.find_all("br"):
        br.replace_with("\n")
    for p in soup.find_all("p"):
        p.insert_after("\n")
    text = soup.get_text(" ", strip=True)
    return text

def fetch_reliefweb(country, start, end):
    url = f"https://api.reliefweb.int/v1/reports?appname=UTDallasCapstone1-01JCBs7"
    limit = 200
    payload = {
        "filter": {
            "operator": "AND",
            "conditions": [
                {"field": "country.name", "value": country},
                {"field": "date.created", "value": {"from": f"{start}T00:00:00+00:00", "to": f"{end}T23:59:59+00:00"}},
                {"field": "language.name", "value": "English"}
            ]
        },
        "profile": "full",
        "limit": limit,
        "offset": 0
    }
    response = requests.post(url, json=payload)
    response.raise_for_status()
    data = response.json().get("data", [])
    reports = []
    for item in data:
        f = item["fields"]
        reports.append({
            "title": f.get("title"),
            "date": f.get("date", {}).get("original", "").replace("T00:00:00+00:00", ""),
            "source": ", ".join([s["name"] for s in f.get("source", [])]) if f.get("source") else "",
            "body": clean_html(f.get("body")),
        })
    content = ""
    for report in reports:
        if not report["title"]:
            continue
        content += f"""
Title: {report['title'].strip()}
Date: {report['date'].strip()}
Source: {report['source'].strip()}
Article Content: {report['body'].strip()}

"""
    return content

def create_prompt_reliefweb(country, start, end):
    content = fetch_reliefweb(country, start, end)
    if not content:
        return f"No ReliefWeb English reports for {country} between {start} and {end}."
    return f""" These are multiple articles for {country} published between {start} and {end} from ReliefWeb API:
{content}
"""

def fetch_gnews(query, start, end, api_key):
    url = "https://gnews.io/api/v4/search"
    all_articles = []
    page = 1
    while True:
        params = {"q": query, "lang": "en", "from": start, "to": end, "max": 100, "page": page, "token": api_key}
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        articles = data.get("articles", [])
        if not articles:
            break
        all_articles.extend(articles)
        if len(articles) < 100:
            break
        page += 1
    if not all_articles:
        return pd.DataFrame(columns=["source", "title", "date", "url", "description"])
    df = pd.json_normalize(all_articles)
    df = df.rename(columns={
        "source.name": "source",
        "title": "title",
        "publishedAt": "date",
        "url": "url",
        "description": "description"
    })
    df["date"] = pd.to_datetime(df["date"]).dt.date
    return df[["source", "title", "date", "url", "description"]]

def create_prompt_gnews(country, start, end):
    api_key = os.getenv("G_NEWS")
    if not api_key:
        return "GNews API key missing."
    df = fetch_gnews(country, start, end, api_key)
    if df.empty:
        return f"No Google News articles for {country} between {start} and {end}."
    content = ""
    for row in df.itertuples(index=False):
        content += f"""
Title: {row.title.strip()}
Date: {row.date}
Source: {row.source.strip()}
Article Content: {row.description.strip()}

"""
    return f""" These are multiple articles for {country} published between {start} and {end} from Google News API:
{content}
"""

OAUTH_LINK = "https://acleddata.com/oauth/token"
URL = "https://acleddata.com/api/acled/read"

def request_oauth(username, password):
    payload = {
        "username": username,
        "password": password,
        "grant_type": "password",
        "client_id": "acled"
    }
    response = requests.post(OAUTH_LINK, data=payload)
    response = response.json()
    return response.get("access_token")

def fetch_fatalities(country, start, end):
    token = request_oauth(os.getenv("USERNAME"), os.getenv("PASSWORD"))
    if not token:
        return pd.DataFrame()
    headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
    params = {"_format": "json", "country": country, "event_date": f"{start}|{end}", "event_date_where": "BETWEEN"}
    response = requests.get(URL, headers=headers, params=params)
    response = response.json()
    df = pd.json_normalize(response.get('data', []))
    return df

def create_prompt_acled(country, start, end):
    df = fetch_fatalities(country, start, end)
    if df.empty:
        return "No recorded fatalities from political violence by ACLED."
    content = ""
    for row in df.itertuples(index=False):
        content += f"""These are events with recorded fatalities by ACLED in {country} between {start} and {end}:
Date: {getattr(row, 'event_date', '')}
Description: {getattr(row, 'notes', '')}
Fatalities: {getattr(row, 'fatalities', '')}
"""
    return content

def get_prompts(country_name, start, end):
    country_code = pycountry.countries.lookup(country_name).alpha_3

    prompt1 = create_prompt_worldbank(country_code, int(start[:4]) - 10, end[:4])
    prompt2 = create_prompt_reliefweb(country_name, start, end)
    prompt3 = create_prompt_gnews(country_name, start, end)
    prompt4 = create_prompt_acled(country_name, start, end)

    text = f"{prompt1}\n{prompt4}\n{prompt2}\n{prompt3}"

    return text

def translate_text(text, target_language):
    deepl_client = deepl.DeepLClient(os.getenv("DEEPL"))
    result = deepl_client.translate_text(text, target_lang=target_language)
    return result

def generate_report_llama(country_name: str, start: str, end: str, text: str, current_model: str, current_lang: str) -> str:
    init_backend()
    global _llama_pipeline, _base_prompt, _llama_pipeline2

    try:
        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        chunks = splitter.create_documents([text])

        embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

        chroma = Chroma.from_documents(documents=chunks, embedding=embedding_model)

        STATIC_QUERY = f"""
Gather comprehensive humanitarian, socioeconomic, and situational information for {country_name}
from datasets and reports published by the World Bank, ReliefWeb, ACLED, and Google News.
Ensure results include both quantitative datasets and narrative situation updates.
"""
        retriever = chroma.as_retriever(search_type="mmr", search_kwargs={"k": 25, "fetch_k": 60, "lambda_mult": 0.7})
        relevant_docs = retriever.invoke(STATIC_QUERY)
        context = "\n\n".join([doc.page_content for doc in relevant_docs])
        messages = [
            {"role": "system", "content": _base_prompt},
            {"role": "user", "content": f"\nRelevant data about {country_name}: {context}"},
        ]

        if current_model == "3B-Llama":
          outputs = _llama_pipeline(messages)
        else:
          outputs = _llama_pipeline2(messages, max_new_tokens=1500)

        result = outputs[0]["generated_text"]

        return result

    except Exception as e:
        error_msg = f"Error in generate_report_llama: {str(e)}"
        print(error_msg)
        import traceback
        traceback.print_exc()
        return f"Error generating report: {str(e)}\n\nPlease check the console for more details."


In [ ]:
from llama_backend import init_backend
from langchain_huggingface import HuggingFaceEmbeddings

init_backend()


In [ ]:
%%writefile app.py
import streamlit as st
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, date
import json
import os
from typing import Dict, List, Optional, Tuple
import random
import pycountry
from llama_backend import generate_report_llama, fetch_indicators, get_prompts, translate_text
import io
from datetime import datetime
from typing import Dict, Any, Optional
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak, Image
from reportlab.lib import colors
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.style import WD_STYLE_TYPE
import streamlit as st
import deepl

TRANSLATIONS = {
    'en': {
        # Navigation
        'nav_home': 'Home',
        'nav_history': 'History',
        'nav_settings': 'Settings',
        'nav_help': 'Help',

        # Main headers
        'app_title': 'NGO Data Helpers',
        'app_subtitle': 'Generate comprehensive humanitarian and development reports for any country',
        'report_preview': 'Report Preview',
        'help_title': 'Help & About',
        'settings_title': 'Settings',
        'history_title': 'Report History',

        # Home page
        'select_country': 'Select Country',
        'select_country_placeholder': 'Choose a country...',
        'select_date_range': 'Select Date Range',
        'start_date': 'Start Date',
        'end_date': 'End Date',
        'build_report': 'Build Report',
        'building': 'Building...',
        'progress_title': 'Progress',

        # Progress steps
        'step_collecting': 'Collecting Data',
        'step_processing': 'Processing',
        'step_summarizing': 'Summarizing',
        'step_ready': 'Ready',

        # Report sections
        'executive_summary': 'Executive Summary',
        'key_events': 'Key Events',
        'trends': 'Trends',
        'risks': 'Risks',
        'data_visualization': 'Data Visualization',
        'monthly_trends': 'Monthly Trends',
        'monthly_comparison': 'Monthly Comparison',
        'download_report': 'Download Report',
        'download_description': 'Download your report in your preferred format:',
        'download_pdf': 'Download PDF',
        'download_docx': 'Download DOCX',
        'back_to_home': 'Back to Home',

        # Help page
        'how_to_use': 'How to Use the App',
        'help_description': 'The NGO Data Helpers app allows you to generate comprehensive humanitarian and development reports for any country within a specified date range.',
        'step1_title': 'Select Country and Dates',
        'step1_desc': 'Choose a country from the dropdown menu and select your desired start and end dates for the report period.',
        'step2_title': 'Build Report',
        'step2_desc': 'Click the "Build Report" button to generate your report. You\'ll see progress indicators as the system processes your request.',
        'step3_title': 'Preview Report',
        'step3_desc': 'Review the generated report with summaries, events, trends, and interactive charts.',
        'step4_title': 'Download Report',
        'step4_desc': 'Once the report is ready, you can preview it and download it in PDF or DOCX format.',
        'features_title': 'Features',
        'feature_reports': 'Comprehensive Reports',
        'feature_reports_desc': 'Get detailed summaries, key events, trends, and risk assessments',
        'feature_charts': 'Data Visualization',
        'feature_charts_desc': 'Interactive charts and graphs to better understand trends',
        'feature_download': 'Multiple Formats',
        'feature_download_desc': 'Download reports in PDF or DOCX format for easy sharing',
        'troubleshooting_title': 'Troubleshooting',
        'error_country_dates': 'Please pick a country and valid dates',
        'error_country_dates_desc': 'Make sure you\'ve selected a country from the dropdown and chosen both start and end dates.',
        'error_build_report': 'Could not build report',
        'error_build_report_desc': 'There was an error processing your request. Try again or check your internet connection.',
        'error_download': 'Download failed',
        'error_download_desc': 'The download couldn\'t be completed. Try again or check your browser\'s download settings.',
        'about_title': 'About',
        'about_desc': 'NGO Data Helpers is designed to assist humanitarian organizations and development workers in generating comprehensive reports quickly and efficiently.',

        # Settings page
        'language_settings': 'Language Settings',
        'model_selection' : 'Model Selection',
        'select_language': 'Select Language:',
        'save_settings': 'Save Settings',
        'settings_saved': 'Settings saved successfully!',
        'language_info': 'Language Information',
        'language_info_desc': 'Complete translation for all interface elements',
        'settings_persistence': 'Settings Persistence',
        'settings_persistence_desc': 'Your language preference is automatically saved and will be remembered the next time you visit the application.',
        'theme_settings': 'Theme Settings',
        'select_theme': 'Select Theme:',
        'light_mode': 'Light Mode',
        'dark_mode': 'Dark Mode',
        'theme_info': 'Theme Information',
        'theme_info_desc': 'Choose your preferred color scheme for the interface',

        # History page
        'no_reports': 'No reports found',
        'no_reports_desc': 'Generate your first report to see it here!',
        'search_reports': 'Search reports:',
        'search_placeholder': 'Search by country, date, or title...',
        'no_matching_reports': 'No reports match your search criteria.',
        'country': 'Country',
        'date_range': 'Date Range',
        'created': 'Created',
        'preview': 'Preview',
        'open': 'Open',
        'delete': 'Delete',
        'history_stats': 'History Statistics',
        'total_reports': 'Total Reports',
        'countries_covered': 'Countries Covered',

        # Common
        'dismiss': 'Dismiss',
        'back': 'Back',
        'version': 'Version 1.0.0',
        'built_with': 'Built with Streamlit'
    },

    'es': {
        # Navigation
        'nav_home': 'Inicio',
        'nav_history': 'Historial',
        'nav_settings': 'Configuración',
        'nav_help': 'Ayuda',

        # Main headers
        'app_title': 'Ayudantes de Datos ONG',
        'app_subtitle': 'Genera informes integrales humanitarios y de desarrollo para cualquier país',
        'report_preview': 'Vista Previa del Informe',
        'help_title': 'Ayuda y Acerca de',
        'settings_title': 'Configuración',
        'history_title': 'Historial de Informes',

        # Home page
        'select_country': 'Seleccionar País',
        'select_country_placeholder': 'Elige un país...',
        'select_date_range': 'Seleccionar Rango de Fechas',
        'start_date': 'Fecha de Inicio',
        'end_date': 'Fecha de Fin',
        'build_report': 'Generar Informe',
        'building': 'Generando...',
        'progress_title': 'Progreso',

        # Progress steps
        'step_collecting': 'Recopilando Datos',
        'step_processing': 'Procesando',
        'step_summarizing': 'Resumiendo',
        'step_ready': 'Listo',

        # Report sections
        'executive_summary': 'Resumen Ejecutivo',
        'key_events': 'Eventos Clave',
        'trends': 'Tendencias',
        'risks': 'Riesgos',
        'data_visualization': 'Visualización de Datos',
        'monthly_trends': 'Tendencias Mensuales',
        'monthly_comparison': 'Comparación Mensual',
        'download_report': 'Descargar Informe',
        'download_description': 'Descarga tu informe en tu formato preferido:',
        'download_pdf': 'Descargar PDF',
        'download_docx': 'Descargar DOCX',
        'back_to_home': 'Volver al Inicio',

        # Help page
        'how_to_use': 'Cómo Usar la Aplicación',
        'help_description': 'La aplicación Ayudantes de Datos ONG te permite generar informes integrales humanitarios y de desarrollo para cualquier país dentro de un rango de fechas específico.',
        'step1_title': 'Seleccionar País y Fechas',
        'step1_desc': 'Elige un país del menú desplegable y selecciona las fechas de inicio y fin deseadas para el período del informe.',
        'step2_title': 'Generar Informe',
        'step2_desc': 'Haz clic en el botón "Generar Informe" para crear tu informe. Verás indicadores de progreso mientras el sistema procesa tu solicitud.',
        'step3_title': 'Vista Previa del Informe',
        'step3_desc': 'Revisa el informe generado con resúmenes, eventos, tendencias y gráficos interactivos.',
        'step4_title': 'Descargar Informe',
        'step4_desc': 'Una vez que el informe esté listo, puedes previsualizarlo y descargarlo en formato PDF o DOCX.',
        'features_title': 'Características',
        'feature_reports': 'Informes Integrales',
        'feature_reports_desc': 'Obtén resúmenes detallados, eventos clave, tendencias y evaluaciones de riesgo',
        'feature_charts': 'Visualización de Datos',
        'feature_charts_desc': 'Gráficos y tablas interactivas para entender mejor las tendencias',
        'feature_download': 'Múltiples Formatos',
        'feature_download_desc': 'Descarga informes en formato PDF o DOCX para compartir fácilmente',
        'troubleshooting_title': 'Solución de Problemas',
        'error_country_dates': 'Por favor selecciona un país y fechas válidas',
        'error_country_dates_desc': 'Asegúrate de haber seleccionado un país del menú desplegable y haber elegido tanto la fecha de inicio como la de fin.',
        'error_build_report': 'No se pudo generar el informe',
        'error_build_report_desc': 'Hubo un error procesando tu solicitud. Inténtalo de nuevo o verifica tu conexión a internet.',
        'error_download': 'Descarga fallida',
        'error_download_desc': 'La descarga no pudo completarse. Inténtalo de nuevo o verifica la configuración de descarga de tu navegador.',
        'about_title': 'Acerca de',
        'about_desc': 'Ayudantes de Datos ONG está diseñado para asistir a organizaciones humanitarias y trabajadores de desarrollo en generar informes integrales de manera rápida y eficiente.',

        # Settings page
        'language_settings': 'Configuración de Idioma',
        'model_selection': 'Selección del Modelo',
        'select_language': 'Seleccionar Idioma:',
        'save_settings': 'Guardar Configuración',
        'settings_saved': '¡Configuración guardada exitosamente!',
        'language_info': 'Información del Idioma',
        'language_info_desc': 'Traducción completa para todos los elementos de la interfaz',
        'settings_persistence': 'Persistencia de Configuración',
        'settings_persistence_desc': 'Tu preferencia de idioma se guarda automáticamente y será recordada la próxima vez que visites la aplicación.',
        'theme_settings': 'Configuración de Tema',
        'select_theme': 'Seleccionar Tema:',
        'light_mode': 'Modo Claro',
        'dark_mode': 'Modo Oscuro',
        'theme_info': 'Información del Tema',
        'theme_info_desc': 'Elige tu esquema de colores preferido para la interfaz',

        # History page
        'no_reports': 'No se encontraron informes',
        'no_reports_desc': '¡Genera tu primer informe para verlo aquí!',
        'search_reports': 'Buscar informes:',
        'search_placeholder': 'Buscar por país, fecha o título...',
        'no_matching_reports': 'Ningún informe coincide con tus criterios de búsqueda.',
        'country': 'País',
        'date_range': 'Rango de Fechas',
        'created': 'Creado',
        'preview': 'Vista Previa',
        'open': 'Abrir',
        'delete': 'Eliminar',
        'history_stats': 'Estadísticas del Historial',
        'total_reports': 'Total de Informes',
        'countries_covered': 'Países Cubiertos',

        # Common
        'dismiss': 'Descartar',
        'back': 'Atrás',
        'version': 'Versión 1.0.0',
        'built_with': 'Construido con Streamlit'
    },

    'fr': {
        # Navigation
        'nav_home': 'Accueil',
        'nav_history': 'Historique',
        'nav_settings': 'Paramètres',
        'nav_help': 'Aide',

        # Main headers
        'app_title': 'Aides aux Données ONG',
        'app_subtitle': 'Générez des rapports complets humanitaires et de développement pour n\'importe quel pays',
        'report_preview': 'Aperçu du Rapport',
        'help_title': 'Aide et À Propos',
        'settings_title': 'Paramètres',
        'history_title': 'Historique des Rapports',

        # Home page
        'select_country': 'Sélectionner un Pays',
        'select_country_placeholder': 'Choisissez un pays...',
        'select_date_range': 'Sélectionner la Plage de Dates',
        'start_date': 'Date de Début',
        'end_date': 'Date de Fin',
        'build_report': 'Générer le Rapport',
        'building': 'Génération...',
        'progress_title': 'Progrès',

        # Progress steps
        'step_collecting': 'Collecte de Données',
        'step_processing': 'Traitement',
        'step_summarizing': 'Résumé',
        'step_ready': 'Prêt',

        # Report sections
        'executive_summary': 'Résumé Exécutif',
        'key_events': 'Événements Clés',
        'trends': 'Tendances',
        'risks': 'Risques',
        'data_visualization': 'Visualisation des Données',
        'monthly_trends': 'Tendances Mensuelles',
        'monthly_comparison': 'Comparaison Mensuelle',
        'download_report': 'Télécharger le Rapport',
        'download_description': 'Téléchargez votre rapport dans votre format préféré :',
        'download_pdf': 'Télécharger PDF',
        'download_docx': 'Télécharger DOCX',
        'back_to_home': 'Retour à l\'Accueil',

        # Help page
        'how_to_use': 'Comment Utiliser l\'Application',
        'help_description': 'L\'application Aides aux Données ONG vous permet de générer des rapports complets humanitaires et de développement pour n\'importe quel pays dans une plage de dates spécifique.',
        'step1_title': 'Sélectionner le Pays et les Dates',
        'step1_desc': 'Choisissez un pays dans le menu déroulant et sélectionnez vos dates de début et de fin souhaitées pour la période du rapport.',
        'step2_title': 'Générer le Rapport',
        'step2_desc': 'Cliquez sur le bouton "Générer le Rapport" pour créer votre rapport. Vous verrez des indicateurs de progression pendant que le système traite votre demande.',
        'step3_title': 'Aperçu du Rapport',
        'step3_desc': 'Examinez le rapport généré avec des résumés, des événements, des tendances et des graphiques interactifs.',
        'step4_title': 'Télécharger le Rapport',
        'step4_desc': 'Une fois le rapport prêt, vous pouvez le prévisualiser et le télécharger au format PDF ou DOCX.',
        'features_title': 'Caractéristiques',
        'feature_reports': 'Rapports Complets',
        'feature_reports_desc': 'Obtenez des résumés détaillés, des événements clés, des tendances et des évaluations des risques',
        'feature_charts': 'Visualisation des Données',
        'feature_charts_desc': 'Graphiques et tableaux interactifs pour mieux comprendre les tendances',
        'feature_download': 'Formats Multiples',
        'feature_download_desc': 'Téléchargez les rapports au format PDF ou DOCX pour un partage facile',
        'troubleshooting_title': 'Dépannage',
        'error_country_dates': 'Veuillez sélectionner un pays et des dates valides',
        'error_country_dates_desc': 'Assurez-vous d\'avoir sélectionné un pays dans le menu déroulant et choisi à la fois les dates de début et de fin.',
        'error_build_report': 'Impossible de générer le rapport',
        'error_build_report_desc': 'Il y a eu une erreur lors du traitement de votre demande. Réessayez ou vérifiez votre connexion internet.',
        'error_download': 'Échec du téléchargement',
        'error_download_desc': 'Le téléchargement n\'a pas pu être complété. Réessayez ou vérifiez les paramètres de téléchargement de votre navigateur.',
        'about_title': 'À Propos',
        'about_desc': 'Aides aux Données ONG est conçu pour aider les organisations humanitaires et les travailleurs du développement à générer des rapports complets rapidement et efficacement.',

        # Settings page
        'language_settings': 'Paramètres de Langue',
        'model_selection': 'Sélection du Modèle',
        'select_language': 'Sélectionner la Langue :',
        'save_settings': 'Enregistrer les Paramètres',
        'settings_saved': 'Paramètres enregistrés avec succès !',
        'language_info': 'Informations sur la Langue',
        'language_info_desc': 'Traduction complète pour tous les éléments de l\'interface',
        'settings_persistence': 'Persistance des Paramètres',
        'settings_persistence_desc': 'Votre préférence de langue est automatiquement sauvegardée et sera mémorisée la prochaine fois que vous visiterez l\'application.',
        'theme_settings': 'Paramètres de Thème',
        'select_theme': 'Sélectionner le Thème :',
        'light_mode': 'Mode Clair',
        'dark_mode': 'Mode Sombre',
        'theme_info': 'Informations sur le Thème',
        'theme_info_desc': 'Choisissez votre schéma de couleurs préféré pour l\'interface',

        # History page
        'no_reports': 'Aucun rapport trouvé',
        'no_reports_desc': 'Générez votre premier rapport pour le voir ici !',
        'search_reports': 'Rechercher des rapports :',
        'search_placeholder': 'Rechercher par pays, date ou titre...',
        'no_matching_reports': 'Aucun rapport ne correspond à vos critères de recherche.',
        'country': 'Pays',
        'date_range': 'Plage de Dates',
        'created': 'Créé',
        'preview': 'Aperçu',
        'open': 'Ouvrir',
        'delete': 'Supprimer',
        'history_stats': 'Statistiques de l\'Historique',
        'total_reports': 'Total des Rapports',
        'countries_covered': 'Pays Couverts',

        # Common
        'dismiss': 'Rejeter',
        'back': 'Retour',
        'version': 'Version 1.0.0',
        'built_with': 'Construit avec Streamlit'
    },

    'ar': {
        # Navigation
        'nav_home': 'الرئيسية',
        'nav_history': 'التاريخ',
        'nav_settings': 'الإعدادات',
        'nav_help': 'المساعدة',

        # Main headers
        'app_title': 'مساعدو بيانات المنظمات غير الحكومية',
        'app_subtitle': 'قم بتوليد تقارير إنسانية وتنموية شاملة لأي بلد',
        'report_preview': 'معاينة التقرير',
        'help_title': 'المساعدة وحول',
        'settings_title': 'الإعدادات',
        'history_title': 'تاريخ التقارير',

        # Home page
        'select_country': 'اختر البلد',
        'select_country_placeholder': 'اختر بلداً...',
        'select_date_range': 'اختر نطاق التاريخ',
        'start_date': 'تاريخ البداية',
        'end_date': 'تاريخ النهاية',
        'build_report': 'إنشاء التقرير',
        'building': 'جاري الإنشاء...',
        'progress_title': 'التقدم',

        # Progress steps
        'step_collecting': 'جمع البيانات',
        'step_processing': 'المعالجة',
        'step_summarizing': 'التلخيص',
        'step_ready': 'جاهز',

        # Report sections
        'executive_summary': 'الملخص التنفيذي',
        'key_events': 'الأحداث الرئيسية',
        'trends': 'الاتجاهات',
        'risks': 'المخاطر',
        'data_visualization': 'تصور البيانات',
        'monthly_trends': 'الاتجاهات الشهرية',
        'monthly_comparison': 'المقارنة الشهرية',
        'download_report': 'تحميل التقرير',
        'download_description': 'قم بتحميل تقريرك بالتنسيق المفضل لديك:',
        'download_pdf': 'تحميل PDF',
        'download_docx': 'تحميل DOCX',
        'back_to_home': 'العودة للرئيسية',

        # Help page
        'how_to_use': 'كيفية استخدام التطبيق',
        'help_description': 'يتيح تطبيق مساعدو بيانات المنظمات غير الحكومية توليد تقارير إنسانية وتنموية شاملة لأي بلد ضمن نطاق تاريخ محدد.',
        'step1_title': 'اختر البلد والتواريخ',
        'step1_desc': 'اختر بلداً من القائمة المنسدلة وحدد تواريخ البداية والنهاية المطلوبة لفترة التقرير.',
        'step2_title': 'إنشاء التقرير',
        'step2_desc': 'انقر على زر "إنشاء التقرير" لتوليد تقريرك. سترى مؤشرات التقدم بينما يعالج النظام طلبك.',
        'step3_title': 'معاينة التقرير',
        'step3_desc': 'راجع التقرير المُولد مع الملخصات والأحداث والاتجاهات والرسوم البيانية التفاعلية.',
        'step4_title': 'تحميل التقرير',
        'step4_desc': 'بمجرد أن يصبح التقرير جاهزاً، يمكنك معاينته وتحميله بتنسيق PDF أو DOCX.',
        'features_title': 'الميزات',
        'feature_reports': 'التقارير الشاملة',
        'feature_reports_desc': 'احصل على ملخصات مفصلة وأحداث رئيسية واتجاهات وتقييمات للمخاطر',
        'feature_charts': 'تصور البيانات',
        'feature_charts_desc': 'رسوم بيانية وجداول تفاعلية لفهم الاتجاهات بشكل أفضل',
        'feature_download': 'تنسيقات متعددة',
        'feature_download_desc': 'قم بتحميل التقارير بتنسيق PDF أو DOCX للمشاركة السهلة',
        'troubleshooting_title': 'استكشاف الأخطاء',
        'error_country_dates': 'يرجى اختيار بلد وتواريخ صحيحة',
        'error_country_dates_desc': 'تأكد من اختيار بلد من القائمة المنسدلة واختيار كل من تاريخ البداية والنهاية.',
        'error_build_report': 'لا يمكن إنشاء التقرير',
        'error_build_report_desc': 'حدث خطأ في معالجة طلبك. حاول مرة أخرى أو تحقق من اتصالك بالإنترنت.',
        'error_download': 'فشل التحميل',
        'error_download_desc': 'لا يمكن إكمال التحميل. حاول مرة أخرى أو تحقق من إعدادات التحميل في متصفحك.',
        'about_title': 'حول',
        'about_desc': 'تم تصميم مساعدو بيانات المنظمات غير الحكومية لمساعدة المنظمات الإنسانية والعاملين في التنمية على توليد تقارير شاملة بسرعة وكفاءة.',

        # Settings page
        'language_settings': 'إعدادات اللغة',
        'model_selection' : 'اختيار النموذج',
        'select_language': 'اختر اللغة:',
        'save_settings': 'حفظ الإعدادات',
        'settings_saved': 'تم حفظ الإعدادات بنجاح!',
        'language_info': 'معلومات اللغة',
        'language_info_desc': 'ترجمة كاملة لجميع عناصر الواجهة',
        'settings_persistence': 'استمرارية الإعدادات',
        'settings_persistence_desc': 'يتم حفظ تفضيل اللغة تلقائياً وسيتم تذكره في المرة القادمة التي تزور فيها التطبيق.',
        'theme_settings': 'إعدادات المظهر',
        'select_theme': 'اختر المظهر:',
        'light_mode': 'الوضع الفاتح',
        'dark_mode': 'الوضع المظلم',
        'theme_info': 'معلومات المظهر',
        'theme_info_desc': 'اختر نظام الألوان المفضل لديك للواجهة',

        # History page
        'no_reports': 'لم يتم العثور على تقارير',
        'no_reports_desc': 'قم بإنشاء تقريرك الأول لتراه هنا!',
        'search_reports': 'البحث في التقارير:',
        'search_placeholder': 'البحث بالبلد أو التاريخ أو العنوان...',
        'no_matching_reports': 'لا توجد تقارير تطابق معايير البحث الخاصة بك.',
        'country': 'البلد',
        'date_range': 'نطاق التاريخ',
        'created': 'تم الإنشاء',
        'preview': 'معاينة',
        'open': 'فتح',
        'delete': 'حذف',
        'history_stats': 'إحصائيات التاريخ',
        'total_reports': 'إجمالي التقارير',
        'countries_covered': 'البلدان المغطاة',

        # Common
        'dismiss': 'رفض',
        'back': 'رجوع',
        'version': 'الإصدار 1.0.0',
        'built_with': 'مبني بـ Streamlit'
    }
}

def get_translation(language: str, key: str) -> str:
    if language not in TRANSLATIONS:
        language = 'en'

    if key not in TRANSLATIONS[language]:
        # Fallback to English if key not found
        return TRANSLATIONS['en'].get(key, key)

    return TRANSLATIONS[language][key]

def get_supported_languages():
    return [
        {'code': 'en', 'name': 'English', 'flag': '🇺🇸'},
        {'code': 'es', 'name': 'Spanish', 'flag': '🇪🇸'},
        {'code': 'fr', 'name': 'French', 'flag': '🇫🇷'},
        {'code': 'ar', 'name': 'Arabic', 'flag': '🇸🇦'}
    ]

class ReportGenerator:

    def __init__(self):
        self.styles = getSampleStyleSheet()
        self._setup_custom_styles()

    def _setup_custom_styles(self):
        self.styles.add(ParagraphStyle(
            name='CustomTitle',
            parent=self.styles['Heading1'],
            fontSize=24,
            spaceAfter=30,
            alignment=1,
            textColor=colors.darkblue
        ))

        self.styles.add(ParagraphStyle(
            name='CustomHeading',
            parent=self.styles['Heading2'],
            fontSize=16,
            spaceAfter=12,
            textColor=colors.darkblue
        ))

        self.styles.add(ParagraphStyle(
            name='CustomBody',
            parent=self.styles['Normal'],
            fontSize=11,
            spaceAfter=6,
            leftIndent=0
        ))

    def generate_pdf(self, report_data: Dict[str, Any], country: str, date_range: Dict[str, str]) -> bytes:
        buffer = io.BytesIO()
        doc = SimpleDocTemplate(buffer, pagesize=A4, rightMargin=72, leftMargin=72, topMargin=72, bottomMargin=18)

        story = []

        # Title
        title = f"NGO Data Helpers Report: {country}"
        story.append(Paragraph(title, self.styles['CustomTitle']))
        story.append(Spacer(1, 12))

        # Subtitle
        subtitle = f"Date Range: {date_range['start_date']} to {date_range['end_date']}"
        story.append(Paragraph(subtitle, self.styles['CustomBody']))
        story.append(Spacer(1, 12))

        # Executive Summary
        story.append(Paragraph("Executive Summary", self.styles['CustomHeading']))
        story.append(Paragraph(report_data.get('summary', 'No summary available.'), self.styles['CustomBody']))
        story.append(Spacer(1, 20))

        # Key Events
        story.append(Paragraph("Key Events", self.styles['CustomHeading']))
        key_events = report_data.get('key_events', [])
        if key_events:
            for event in key_events:
                story.append(Paragraph(event, self.styles['CustomBody']))
        else:
            story.append(Paragraph("No key events available.", self.styles['CustomBody']))
        story.append(Spacer(1, 20))

        # Trends
        story.append(Paragraph("Trends", self.styles['CustomHeading']))
        trends = report_data.get('trends', [])
        if trends:
            for trend in trends:
                story.append(Paragraph(trend, self.styles['CustomBody']))
        else:
            story.append(Paragraph("No trends available.", self.styles['CustomBody']))
        story.append(Spacer(1, 20))

        # Risks
        story.append(Paragraph("Risks", self.styles['CustomHeading']))
        risks = report_data.get('risks', [])
        if risks:
            for risk in risks:
                story.append(Paragraph(risk, self.styles['CustomBody']))
        else:
            story.append(Paragraph("No risks identified.", self.styles['CustomBody']))
        story.append(Spacer(1, 20))

        # Charts
        for name, chart_bytes in st.session_state.charts.items():
          story.append(Paragraph(f"<b>{name}</b>", self.styles['Heading2']))
          story.append(Spacer(1, 0.1 * inch))
          story.append(Image(io.BytesIO(chart_bytes), width=5.5*inch, height=3.0*inch))
          story.append(Spacer(1, 0.2 * inch))

        # Data Sources
        story.append(Paragraph("Data Sources", self.styles['CustomHeading']))
        sources = report_data.get('sources', [])
        if sources:
            for source in sources:
                story.append(Paragraph(f"• {source}", self.styles['CustomBody']))
        else:
            story.append(Paragraph("• ACLED (Armed Conflict Location & Event Data Project)", self.styles['CustomBody']))
            story.append(Paragraph("• GDELT (Global Database of Events, Language, and Tone)", self.styles['CustomBody']))
            story.append(Paragraph("• ReliefWeb (Humanitarian Information Service)", self.styles['CustomBody']))
            story.append(Paragraph("• World Bank Open Data", self.styles['CustomBody']))

        doc.build(story)
        buffer.seek(0)
        return buffer.getvalue()

    def generate_docx(self, report_data: Dict[str, Any], country: str, date_range: Dict[str, str]) -> bytes:
        doc = Document()

        # Title
        title = doc.add_heading(f"NGO Data Helpers Report: {country}", 0)
        title.alignment = WD_ALIGN_PARAGRAPH.CENTER

        # Subtitle
        subtitle = doc.add_paragraph(f"Date Range: {date_range['start_date']} to {date_range['end_date']}")
        subtitle.alignment = WD_ALIGN_PARAGRAPH.CENTER

        # Add some space
        doc.add_paragraph()

        # Executive Summary
        doc.add_heading("Executive Summary", level=1)
        doc.add_paragraph(report_data.get('summary', 'No summary available.'))
        doc.add_paragraph()

        # Key Events
        doc.add_heading("Key Events", level=1)
        key_events = report_data.get('key_events', [])
        if key_events:
            for event in key_events:
                doc.add_paragraph(event)
        else:
            doc.add_paragraph("No key events available.")
        doc.add_paragraph()

        # Trends
        doc.add_heading("Trends", level=1)
        trends = report_data.get('trends', [])
        if trends:
            for trend in trends:
                doc.add_paragraph(trend)
        else:
            doc.add_paragraph("No trends available.")
        doc.add_paragraph()

        # Risks
        doc.add_heading("Risks", level=1)
        risks = report_data.get('risks', [])
        if risks:
            for risk in risks:
                doc.add_paragraph(risk)
        else:
            doc.add_paragraph("No risks identified.")
        doc.add_paragraph()

        # Charts
        for name, chart_bytes in st.session_state.charts.items():
          doc.add_heading(name, level=1)
          doc.add_picture(io.BytesIO(chart_bytes), width=Inches(5.5), height=Inches(3.0))
          doc.add_paragraph()

        # Data Sources
        doc.add_heading("Data Sources", level=1)
        sources = report_data.get('sources', [])
        if sources:
            for source in sources:
                doc.add_paragraph(f"• {source}")
        else:
            doc.add_paragraph("• ACLED (Armed Conflict Location & Event Data Project)")
            doc.add_paragraph("• GDELT (Global Database of Events, Language, and Tone)")
            doc.add_paragraph("• ReliefWeb (Humanitarian Information Service)")
            doc.add_paragraph("• World Bank Open Data")

        buffer = io.BytesIO()
        doc.save(buffer)
        buffer.seek(0)
        return buffer.getvalue()


def download_pdf_report(report_data: Dict[str, Any], country: str, date_range: Dict[str, str]) -> None:
    try:
        generator = ReportGenerator()
        pdf_bytes = generator.generate_pdf(report_data, country, date_range)

        filename = f"NGO_Report_{country.replace(' ', '_')}_{date_range['start_date']}_to_{date_range['end_date']}.pdf"

        st.download_button(
            label="📄 Download PDF Report",
            data=pdf_bytes,
            file_name=filename,
            mime="application/pdf",
            type="primary"
        )
    except Exception as e:
        st.error(f"Error generating PDF: {str(e)}")


def download_docx_report(report_data: Dict[str, Any], country: str, date_range: Dict[str, str]) -> None:
    try:
        generator = ReportGenerator()
        docx_bytes = generator.generate_docx(report_data, country, date_range)

        filename = f"NGO_Report_{country.replace(' ', '_')}_{date_range['start_date']}_to_{date_range['end_date']}.docx"

        st.download_button(
            label="📝 Download DOCX Report",
            data=docx_bytes,
            file_name=filename,
            mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document"
        )
    except Exception as e:
        st.error(f"Error generating DOCX: {str(e)}")

st.set_page_config(
    page_title="NGO Data Helpers",
    page_icon="🌍",
    layout="wide",
    initial_sidebar_state="expanded"
)

def get_css():
    return """
    <style>
        .main-header {
            font-size: 2.5rem;
            font-weight: bold;
            text-align: center;
            margin-bottom: 2rem;
        }
        .section-header {
            font-size: 1.5rem;
            font-weight: 600;
            margin-top: 2rem;
            margin-bottom: 1rem;
        }
        .metric-card {
            padding: 1rem;
            border-radius: 0.5rem;
            border-left: 4px solid #3b82f6;
            margin-bottom: 1rem;
        }
        .error-message {
            background-color: #fef2f2;
            border: 1px solid #fecaca;
            color: #dc2626;
            padding: 1rem;
            border-radius: 0.5rem;
            margin-bottom: 1rem;
        }
        .success-message {
            background-color: #f0fdf4;
            border: 1px solid #bbf7d0;
            color: #16a34a;
            padding: 1rem;
            border-radius: 0.5rem;
            margin-bottom: 1rem;
        }
    </style>
    """

if 'current_page' not in st.session_state:
    st.session_state.current_page = 'home'
if 'selected_country' not in st.session_state:
    st.session_state.selected_country = None
if 'date_range' not in st.session_state:
    st.session_state.date_range = None
if 'report_data' not in st.session_state:
    st.session_state.report_data = None
if 'is_loading' not in st.session_state:
    st.session_state.is_loading = False
if 'progress_steps' not in st.session_state:
    st.session_state.progress_steps = [
        {'id': 'collecting', 'label': 'step_collecting', 'completed': False, 'active': False},
        {'id': 'processing', 'label': 'step_processing', 'completed': False, 'active': False},
        {'id': 'summarizing', 'label': 'step_summarizing', 'completed': False, 'active': False},
        {'id': 'ready', 'label': 'step_ready', 'completed': False, 'active': False}
    ]
if 'current_step' not in st.session_state:
    st.session_state.current_step = 0
if 'error_message' not in st.session_state:
    st.session_state.error_message = None
if 'current_language' not in st.session_state:
    st.session_state.current_language = 'en'
if 'saved_reports' not in st.session_state:
    st.session_state.saved_reports = []
if 'charts' not in st.session_state:
    st.session_state.charts = {}
if 'current_model' not in st.session_state:
    st.session_state.current_model = '3B-Llama'

COUNTRIES = [
            {'code': 'AF', 'name': 'Afghanistan'},
            {'code': 'AL', 'name': 'Albania'},
            {'code': 'DZ', 'name': 'Algeria'},
            {'code': 'AO', 'name': 'Angola'},
            {'code': 'AR', 'name': 'Argentina'},
            {'code': 'AM', 'name': 'Armenia'},
            {'code': 'AU', 'name': 'Australia'},
            {'code': 'AT', 'name': 'Austria'},
            {'code': 'AZ', 'name': 'Azerbaijan'},
            {'code': 'BD', 'name': 'Bangladesh'},
            {'code': 'BR', 'name': 'Brazil'},
            {'code': 'CA', 'name': 'Canada'},
            {'code': 'CN', 'name': 'China'},
            {'code': 'FR', 'name': 'France'},
            {'code': 'DE', 'name': 'Germany'},
            {'code': 'IN', 'name': 'India'},
            {'code': 'IT', 'name': 'Italy'},
            {'code': 'JP', 'name': 'Japan'},
            {'code': 'MX', 'name': 'Mexico'},
            {'code': 'NG', 'name': 'Nigeria'},
            {'code': 'PK', 'name': 'Pakistan'},
            {'code': 'RU', 'name': 'Russia'},
            {'code': 'SA', 'name': 'Saudi Arabia'},
            {'code': 'ZA', 'name': 'South Africa'},
            {'code': 'KR', 'name': 'South Korea'},
            {'code': 'ES', 'name': 'Spain'},
            {'code': 'TR', 'name': 'Turkey'},
            {'code': 'GB', 'name': 'United Kingdom'},
            {'code': 'US', 'name': 'United States'},
        ]


st.markdown(get_css(), unsafe_allow_html=True)


def show_error(message: str):
    st.session_state.error_message = message

def clear_error():
    st.session_state.error_message = None

def build_report():
    if not st.session_state.selected_country or not st.session_state.date_range:
        show_error(get_translation(st.session_state.current_language, 'error_country_dates'))
        return

    st.session_state.is_loading = True
    st.session_state.current_step = 0

    progress_container = st.container()
    with progress_container:
        progress_bar = st.progress(0)
        status_text = st.empty()
        detailed_status = st.empty()

    try:
        status_text.text(f"Status: {get_translation(st.session_state.current_language, 'step_collecting')}")

        country_name = st.session_state.selected_country['name']
        start_date = st.session_state.date_range['start_date']
        end_date = st.session_state.date_range['end_date']
        progress_bar.progress(0.25)
        detailed_status.info(f"📥 Collecting data for {country_name} ({start_date} to {end_date})")
        text = get_prompts(country_name, start_date, end_date)
        progress_bar.progress(0.5)

        status_text.text(f"Status: {get_translation(st.session_state.current_language, 'step_processing')}")
        progress_bar.progress(0.75)
        detailed_status.warning("⏳ This may take 1-2 minutes. Please wait...")

        output_text = generate_report_llama(
            country_name,
            start_date,
            end_date,
            text,
            st.session_state.current_model,
            st.session_state.current_language
        )

        progress_bar.progress(0.95)
        detailed_status.success("✅ Report generated successfully!")


        report_sections = parse_llama_output(output_text)

        st.session_state.report_data = report_sections

        status_text.text(f"Status: {get_translation(st.session_state.current_language, 'step_ready')}")
        progress_bar.progress(1.0)
        detailed_status.success("✅ Report is ready for preview!")

    except Exception as e:
        st.error(f"❌ Error generating report: {str(e)}")
        detailed_status.error(f"Error details: {str(e)}")

        st.session_state.report_data = {
            "summary": f"Error generating report: {str(e)}",
            "key_events": ["Error occurred during report generation"],
            "trends": ["Unable to extract trends due to error"],
            "risks": ["Unable to assess risks due to error"],
            "chart_data": []
        }

        st.session_state.is_loading = False
        return

    report_id = str(datetime.now().timestamp())
    saved_report = {
        'id': report_id,
        'country': st.session_state.selected_country,
        'date_range': st.session_state.date_range,
        'report_data': st.session_state.report_data,
        'created_at': datetime.now().isoformat(),
        'title': f"{st.session_state.selected_country['name']} Report - {st.session_state.date_range['start_date']} to {st.session_state.date_range['end_date']}"
    }

    st.session_state.saved_reports.append(saved_report)

    st.session_state.is_loading = False
    st.session_state.current_page = 'preview'

    import time
    time.sleep(1)
    st.rerun()


def parse_llama_output(output_text: str) -> Dict[str, Any]:
    report = {
        "summary": "",
        "key_events": [],
        "trends": [],
        "risks": [],
        "chart_data": []
    }

    try:
        sections = output_text.split("**")

        current_section = None
        for section in sections:
            section = section.strip()

            if "Executive Summary" in section:
                current_section = "summary"
            elif "Key Events" in section:
                current_section = "key_events"
            elif "Trends" in section:
                current_section = "trends"
            elif "Risks" in section:
                current_section = "risks"
            elif current_section and section:
                lines = section.splitlines()

                if current_section == "summary":
                    report["summary"] = ' '.join(lines)
                elif current_section in ["key_events", "trends", "risks"]:
                    for line in lines:
                        cleaned = line.lstrip(':•-–—*').strip()
                        cleaned = cleaned.lstrip('0123456789.').strip()
                        if cleaned and len(cleaned) > 10:
                           report[current_section].append(cleaned)

    except Exception as e:
        # Fallback: use entire output as summary
        report["summary"] = output_text
        report["key_events"] = ["See full report in summary section"]
        report["trends"] = ["See full report in summary section"]
        report["risks"] = ["See full report in summary section"]

    return report

def render_home_page():
    st.markdown(f'<h1 class="main-header">🌍 {get_translation(st.session_state.current_language, "app_title")}</h1>', unsafe_allow_html=True)
    st.markdown(f'<p style="text-align: center; color: #6b7280; margin-bottom: 2rem;">{get_translation(st.session_state.current_language, "app_subtitle")}</p>', unsafe_allow_html=True)

    # Error message
    if st.session_state.error_message:
        st.markdown(f'<div class="error-message">⚠️ {st.session_state.error_message}</div>', unsafe_allow_html=True)
        if st.button(get_translation(st.session_state.current_language, "dismiss"), key="dismiss_error"):
            clear_error()

    # Main form
    with st.container():
        col1, col2, col3 = st.columns([1, 1, 1])

        with col1:
            st.markdown(f'<h3 class="section-header">📍 {get_translation(st.session_state.current_language, "select_country")}</h3>', unsafe_allow_html=True)
            country_options = [''] + [f"{c['name']} ({c['code']})" for c in COUNTRIES]
            selected_country_str = st.selectbox(
                get_translation(st.session_state.current_language, "select_country_placeholder"),
                options=country_options,
                index=0,
                disabled=st.session_state.is_loading,
                key="country_select"
            )

            if selected_country_str:
                country_code = selected_country_str.split('(')[1].split(')')[0]
                country_name = selected_country_str.split(' (')[0]
                st.session_state.selected_country = {'code': country_code, 'name': country_name}

        with col2:
            st.markdown(f'<h3 class="section-header">📅 {get_translation(st.session_state.current_language, "select_date_range")}</h3>', unsafe_allow_html=True)
            col_start, col_end = st.columns(2)

            with col_start:
                start_date = st.date_input(
                    f"{get_translation(st.session_state.current_language, 'start_date')}:",
                    value=date(2024, 5, 1),
                    disabled=st.session_state.is_loading,
                    key="start_date"
                )

            with col_end:
                end_date = st.date_input(
                    f"{get_translation(st.session_state.current_language, 'end_date')}:",
                    value=date(2024, 5, 31),
                    disabled=st.session_state.is_loading,
                    key="end_date"
                )

            if start_date and end_date:
                st.session_state.date_range = {
                    'start_date': start_date.strftime('%Y-%m-%d'),
                    'end_date': end_date.strftime('%Y-%m-%d')
                }
    with col3:
      st.markdown(f'<h3 class="section-header">{get_translation(st.session_state.current_language, "model_selection")}</h3>', unsafe_allow_html=True)

      selected_model = st.selectbox(
          label=get_translation(st.session_state.current_language, "model_selection"),
          options=["3B-Llama", "8B-Llama"]
      )

    st.session_state.current_model = selected_model

    # Action buttons
    st.markdown('<br>', unsafe_allow_html=True)
    col1, col2, col3 = st.columns([1, 1, 1])

    with col2:
        can_build = st.session_state.selected_country and st.session_state.date_range and not st.session_state.is_loading
        button_text = get_translation(st.session_state.current_language, "building") if st.session_state.is_loading else get_translation(st.session_state.current_language, "build_report")
        if st.button(
            f"🚀 {button_text}",
            disabled=not can_build,
            type="primary",
            use_container_width=True
        ):
            build_report()

    # Progress section
    if st.session_state.is_loading:
        st.markdown(f'<h3 class="section-header">📊 {get_translation(st.session_state.current_language, "progress_title")}</h3>', unsafe_allow_html=True)

        # Progress steps
        cols = st.columns(4)
        for i, step in enumerate(st.session_state.progress_steps):
            with cols[i]:
                step_text = get_translation(st.session_state.current_language, step['label'])
                if step['completed']:
                    st.markdown(f"✅ **{step_text}**")
                elif step['active']:
                    st.markdown(f"⏳ **{step_text}**")
                else:
                    st.markdown(f"⏸️ {step_text}")

def render_report_preview():
    if not st.session_state.report_data:
        st.error("No report data available. Please generate a report first.")
        if st.button(f"← {get_translation(st.session_state.current_language, 'back_to_home')}"):
            st.session_state.current_page = 'home'
            st.rerun()
        return

    # Header
    col1, col2 = st.columns([3, 1])
    with col1:
        st.markdown(f'<h1 class="main-header">📊 {get_translation(st.session_state.current_language, "report_preview")}</h1>', unsafe_allow_html=True)
    with col2:
        if st.button(f"← {get_translation(st.session_state.current_language, 'back_to_home')}", use_container_width=True):
            st.session_state.current_page = 'home'
            st.rerun()

    report = st.session_state.report_data
    cur_lang = st.session_state.current_language
    # Executive Summary
    st.markdown(f'<h2 class="section-header">📋 {get_translation(st.session_state.current_language, "executive_summary")}</h2>', unsafe_allow_html=True)
    st.markdown(f'<div class="metric-card">{report["summary"] if cur_lang == 'en' else translate_text(report["summary"], cur_lang)}</div>', unsafe_allow_html=True)

    # Key Events
    st.markdown(f'<h2 class="section-header">📅 {get_translation(st.session_state.current_language, "key_events")}</h2>', unsafe_allow_html=True)
    for event in report['key_events']:
        st.markdown(event if cur_lang == 'en' else translate_text(event, cur_lang))

    # Trends
    st.markdown(f'<h2 class="section-header">📈 {get_translation(st.session_state.current_language, "trends")}</h2>', unsafe_allow_html=True)
    for trend in report['trends']:
        st.markdown(trend if cur_lang == 'en' else translate_text(trend, cur_lang))

    # Risks
    st.markdown(f'<h2 class="section-header">⚠️ {get_translation(st.session_state.current_language, "risks")}</h2>', unsafe_allow_html=True)
    for risk in report['risks']:
        st.markdown(risk if cur_lang == 'en' else translate_text(risk, cur_lang))

    # Charts
    country_name = st.session_state.selected_country['name']
    start_date = st.session_state.date_range['start_date']
    end_date = st.session_state.date_range['end_date']

    st.markdown(f'<h2 class="section-header">📊 {get_translation(st.session_state.current_language, "data_visualization")}</h2>', unsafe_allow_html=True)
    df = fetch_indicators(pycountry.countries.lookup(country_name).alpha_3, start_date[:4], str(int(end_date[:4]) - 10))
    valid_cols = [col for col in df.columns if df[col].notna().any()]

    def render_chart_as_image(df, col_name):
      buf = io.BytesIO()
      plt.figure(figsize=(6, 4))
      plt.plot(df.index.astype(str), df[col_name], color="#FF0000")
      plt.title(col_name)
      plt.xlabel("Year")
      plt.ylabel("Value")
      plt.xticks(rotation=45, ha="right")
      plt.tight_layout()
      plt.savefig(buf, format="png", bbox_inches="tight")
      buf.seek(0)
      plt.close()
      return buf

    if not valid_cols:
      st.warning("No valid indicator data available.")
    else:
      charts = {}
      for cols in valid_cols:
        charts[cols] = render_chart_as_image(df, cols).getvalue()
      st.session_state.charts = charts
      selected_cols = st.multiselect(
          "Select indicators to visualize:",
          options=valid_cols
      )

    cols = st.columns(2)
    for idx, col in enumerate(selected_cols):
        with cols[idx % 2]:
            st.subheader(col)
            chart = render_chart_as_image(df, col)
            st.image(chart, use_container_width=True)

    # Download section
    st.markdown(f'<h2 class="section-header">💾 {get_translation(st.session_state.current_language, "download_report")}</h2>', unsafe_allow_html=True)
    st.markdown(f'<p style="color: #6b7280;">{get_translation(st.session_state.current_language, "download_description")}</p>', unsafe_allow_html=True)

    col1, col2 = st.columns(2)

    with col1:
        # Generate and download PDF
        download_pdf_report(
            st.session_state.report_data,
            st.session_state.selected_country['name'],
            st.session_state.date_range
        )

    with col2:
        # Generate and download DOCX
        download_docx_report(
            st.session_state.report_data,
            st.session_state.selected_country['name'],
            st.session_state.date_range
        )

def render_help_page():
    st.markdown(f'<h1 class="main-header">❓ {get_translation(st.session_state.current_language, "help_title")}</h1>', unsafe_allow_html=True)

    st.markdown(f'<h2 class="section-header">{get_translation(st.session_state.current_language, "how_to_use")}</h2>', unsafe_allow_html=True)
    st.markdown(f'<p style="color: #6b7280; margin-bottom: 1.5rem;">{get_translation(st.session_state.current_language, "help_description")}</p>', unsafe_allow_html=True)

    # Steps
    steps_data = [
        ("📍", "step1_title", "step1_desc"),
        ("🚀", "step2_title", "step2_desc"),
        ("📊", "step3_title", "step3_desc"),
        ("💾", "step4_title", "step4_desc")
    ]

    for icon, title_key, desc_key in steps_data:
        st.markdown(f'<div class="metric-card"><strong>{icon} {get_translation(st.session_state.current_language, title_key)}</strong><br>{get_translation(st.session_state.current_language, desc_key)}</div>', unsafe_allow_html=True)

    # Features
    st.markdown(f'<h2 class="section-header">{get_translation(st.session_state.current_language, "features_title")}</h2>', unsafe_allow_html=True)
    features_data = [
        ("📋", "feature_reports", "feature_reports_desc"),
        ("📊", "feature_charts", "feature_charts_desc"),
        ("💾", "feature_download", "feature_download_desc")
    ]

    for icon, title_key, desc_key in features_data:
        st.markdown(f'<div class="metric-card"><strong>{icon} {get_translation(st.session_state.current_language, title_key)}</strong><br>{get_translation(st.session_state.current_language, desc_key)}</div>', unsafe_allow_html=True)

    # Troubleshooting
    st.markdown(f'<h2 class="section-header">{get_translation(st.session_state.current_language, "troubleshooting_title")}</h2>', unsafe_allow_html=True)
    troubleshooting_data = [
        ("error_country_dates", "error_country_dates_desc"),
        ("error_build_report", "error_build_report_desc"),
        ("error_download", "error_download_desc")
    ]

    for error_key, solution_key in troubleshooting_data:
        st.markdown(f'<div class="metric-card"><strong>⚠️ "{get_translation(st.session_state.current_language, error_key)}"</strong><br>{get_translation(st.session_state.current_language, solution_key)}</div>', unsafe_allow_html=True)

    # About
    st.markdown(f'<h2 class="section-header">{get_translation(st.session_state.current_language, "about_title")}</h2>', unsafe_allow_html=True)
    st.markdown(f'<p style="color: #6b7280;">{get_translation(st.session_state.current_language, "about_desc")}</p>', unsafe_allow_html=True)

def render_settings_page():
    st.markdown(f'<h1 class="main-header">⚙️ {get_translation(st.session_state.current_language, "settings_title")}</h1>', unsafe_allow_html=True)

    # Language Settings
    st.markdown(f'<h2 class="section-header">{get_translation(st.session_state.current_language, "language_settings")}</h2>', unsafe_allow_html=True)

    languages = get_supported_languages()

    language_options = [f"{lang['flag']} {lang['name']}" for lang in languages]
    current_lang = next((lang for lang in languages if lang['code'] == st.session_state.current_language), languages[0])
    current_index = languages.index(current_lang)

    selected_language = st.selectbox(
        f"{get_translation(st.session_state.current_language, 'select_language')}",
        options=language_options,
        index=current_index,
        key="language_select"
    )

    if selected_language:
        selected_code = languages[language_options.index(selected_language)]['code']
        if selected_code != st.session_state.current_language:
            st.session_state.current_language = selected_code
            st.rerun()

    # Theme Information
    st.markdown(f'<h2 class="section-header">{get_translation(st.session_state.current_language, "theme_settings")}</h2>', unsafe_allow_html=True)
    st.info("🎨 " + get_translation(st.session_state.current_language, "theme_info_desc") + " Use the hamburger menu (☰) in the top-right corner to switch between light and dark themes.")

    # Language Information
    st.markdown(f'<h2 class="section-header">{get_translation(st.session_state.current_language, "language_info")}</h2>', unsafe_allow_html=True)

    for lang in languages:
        st.markdown(f'<div class="metric-card"><strong>{lang["flag"]} {lang["name"]}</strong><br>{get_translation(st.session_state.current_language, "language_info_desc")}</div>', unsafe_allow_html=True)

def render_history_page():
    st.markdown(f'<h1 class="main-header">📚 {get_translation(st.session_state.current_language, "history_title")}</h1>', unsafe_allow_html=True)

    if not st.session_state.saved_reports:
        st.markdown(f'<div class="metric-card" style="text-align: center; padding: 3rem;"><h3 style="color: #6b7280;">{get_translation(st.session_state.current_language, "no_reports")}</h3><p style="color: #9ca3af;">{get_translation(st.session_state.current_language, "no_reports_desc")}</p></div>', unsafe_allow_html=True)
        if st.button(f"← {get_translation(st.session_state.current_language, 'back_to_home')}"):
            st.session_state.current_page = 'home'
            st.rerun()
        return

    # Search functionality
    search_term = st.text_input(f"🔍 {get_translation(st.session_state.current_language, 'search_reports')}", placeholder=get_translation(st.session_state.current_language, "search_placeholder"))

    # Filter reports
    filtered_reports = st.session_state.saved_reports
    if search_term:
        filtered_reports = [
            report for report in st.session_state.saved_reports
            if (search_term.lower() in report['country']['name'].lower() or
                search_term.lower() in report['title'].lower() or
                search_term in report['date_range']['start_date'] or
                search_term in report['date_range']['end_date'])
        ]

    if not filtered_reports:
        st.warning(get_translation(st.session_state.current_language, "no_matching_reports"))
        return

    # Display reports
    for i, report in enumerate(filtered_reports):
        with st.expander(f"📄 {report['title']}", expanded=False):
            col1, col2 = st.columns([3, 1])

            with col1:
                st.markdown(f"**{get_translation(st.session_state.current_language, 'country')}:** {report['country']['name']}")
                st.markdown(f"**{get_translation(st.session_state.current_language, 'date_range')}:** {report['date_range']['start_date']} to {report['date_range']['end_date']}")
                st.markdown(f"**{get_translation(st.session_state.current_language, 'created')}:** {datetime.fromisoformat(report['created_at']).strftime('%Y-%m-%d %H:%M')}")

                # Preview
                st.markdown(f"**{get_translation(st.session_state.current_language, 'preview')}:**")
                st.markdown(f"_{report['report_data']['summary'][:150]}..._")

            with col2:
                if st.button(f"👁️ {get_translation(st.session_state.current_language, 'open')}", key=f"open_{i}"):
                    st.session_state.report_data = report['report_data']
                    st.session_state.current_page = 'preview'
                    st.rerun()

                if st.button(f"🗑️ {get_translation(st.session_state.current_language, 'delete')}", key=f"delete_{i}"):
                    st.session_state.saved_reports.remove(report)
                    st.rerun()

    # Statistics
    st.markdown(f'<h2 class="section-header">📊 {get_translation(st.session_state.current_language, "history_stats")}</h2>', unsafe_allow_html=True)

    col1, col2, col3 = st.columns(3)

    with col1:
        st.metric(get_translation(st.session_state.current_language, "total_reports"), len(st.session_state.saved_reports))

    with col2:
        unique_countries = len(set(report['country']['code'] for report in st.session_state.saved_reports))
        st.metric(get_translation(st.session_state.current_language, "countries_covered"), unique_countries)

# Sidebar navigation
with st.sidebar:
    st.markdown(f'<h2 style="color: #1e293b;">🌍 {get_translation(st.session_state.current_language, "app_title")}</h2>', unsafe_allow_html=True)

    pages = [
        ("🏠", "nav_home", "home"),
        ("📚", "nav_history", "history"),
        ("⚙️", "nav_settings", "settings"),
        ("❓", "nav_help", "help")
    ]

    for icon, name_key, page_key in pages:
        if st.button(f"{icon} {get_translation(st.session_state.current_language, name_key)}", use_container_width=True, key=f"nav_{page_key}"):
            st.session_state.current_page = page_key
            st.rerun()

    st.markdown("---")
    st.markdown(f'<p style="color: #6b7280; font-size: 0.875rem;">{get_translation(st.session_state.current_language, "version")}<br>{get_translation(st.session_state.current_language, "built_with")}</p>', unsafe_allow_html=True)

# Main content area
if st.session_state.current_page == 'home':
    render_home_page()
elif st.session_state.current_page == 'preview':
    render_report_preview()
elif st.session_state.current_page == 'help':
    render_help_page()
elif st.session_state.current_page == 'settings':
    render_settings_page()
elif st.session_state.current_page == 'history':
    render_history_page()

In [ ]:
!pkill ngrok

In [ ]:
from pyngrok import ngrok
import threading
import time
import subprocess
from google.colab import userdata
import os

ngrok.set_auth_token(userdata.get('NGROK'))


public_url = ngrok.connect(8501)

def run_streamlit():
    env = os.environ.copy()
    env["DEEPL"] = userdata.get('DEEPL')
    env["G_NEWS"] = userdata.get('G_NEWS')
    env["HF_TOKEN"] = userdata.get('HF_TOKEN')
    env["PASSWORD"] = userdata.get('PASSWORD')
    env["USERNAME"] = userdata.get('USERNAME')
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8501"],
                   env=env)

thread = threading.Thread(target=run_streamlit)
thread.start()
print(public_url.public_url)
